# **ЛР №3**: Формирование данных для машинного обучения в формате NumPy на основе датасета Semantic3D

## **Ход работы**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def process_semantic3d_data(points_file, labels_file, output_file='semantic3d_dataset.npy'):
    try:
        with open(points_file, 'r') as f:
            first_line = f.readline().strip().split()
            num_columns = len(first_line)
        
        if num_columns == 7:
            data = np.genfromtxt(points_file, dtype=np.float32, comments=None, autostrip=True)
            x = data[:, 0]
            y = data[:, 1]
            z = data[:, 2]
            r = data[:, 3]
            g = data[:, 4]
            b = data[:, 5]
            intensity = data[:, 6]
        elif num_columns == 6:
            data = np.genfromtxt(points_file, dtype=np.float32, comments=None, autostrip=True)
            x = data[:, 0]
            y = data[:, 1]
            z = data[:, 2]
            r = data[:, 3]
            g = data[:, 4]
            b = data[:, 5]
            intensity = np.zeros(len(data), dtype=np.float32)
        elif num_columns == 4:
            data = np.genfromtxt(points_file, dtype=np.float32, comments=None, autostrip=True)
            x = data[:, 0]
            y = data[:, 1]
            z = data[:, 2]
            intensity = data[:, 3]
            r = g = b = np.zeros(len(data), dtype=np.float32)
        else:
            raise ValueError(f"Неподдерживаемый формат файла с {num_columns} столбцами")
            
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        print("Попытка ручной загрузки данных...")
        x, y, z, r, g, b, intensity = [], [], [], [], [], [], []
        
        with open(points_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) not in (4, 6, 7):
                    continue
                
                vals = list(map(float, parts))     
                if len(vals) == 7:
                    x.append(vals[0])
                    y.append(vals[1])
                    z.append(vals[2])
                    r.append(vals[3])
                    g.append(vals[4])
                    b.append(vals[5])
                    intensity.append(vals[6])
                elif len(vals) == 6:
                    x.append(vals[0])
                    y.append(vals[1])
                    z.append(vals[2])
                    r.append(vals[3])
                    g.append(vals[4])
                    b.append(vals[5])
                    intensity.append(0.0)
                elif len(vals) == 4:
                    x.append(vals[0])
                    y.append(vals[1])
                    z.append(vals[2])
                    intensity.append(vals[3])
                    r.append(0.0)
                    g.append(0.0)
                    b.append(0.0)
        
        x = np.array(x, dtype=np.float32)
        y = np.array(y, dtype=np.float32)
        z = np.array(z, dtype=np.float32)
        r = np.array(r, dtype=np.float32)
        g = np.array(g, dtype=np.float32)
        b = np.array(b, dtype=np.float32)
        intensity = np.array(intensity, dtype=np.float32)

    labels = np.loadtxt(labels_file, dtype=np.int32)
    
    min_len = min(len(x), len(labels))
    x, y, z = x[:min_len], y[:min_len], z[:min_len]
    r, g, b = r[:min_len], g[:min_len], b[:min_len]
    intensity = intensity[:min_len]
    labels = labels[:min_len]
    
    r_norm = r / 255.0
    g_norm = g / 255.0
    b_norm = b / 255.0
    
    x_mean, x_std = np.mean(x), np.std(x)
    y_mean, y_std = np.mean(y), np.std(y)
    z_mean, z_std = np.mean(z), np.std(z)
    
    if x_std > 0: x_norm = (x - x_mean) / x_std
    else: x_norm = x - x_mean
    
    if y_std > 0: y_norm = (y - y_mean) / y_std
    else: y_norm = y - y_mean
    
    if z_std > 0: z_norm = (z - z_mean) / z_std
    else: z_norm = z - z_mean
    
    intensity_min, intensity_max = np.min(intensity), np.max(intensity)
    if intensity_max > intensity_min:
        intensity_norm = (intensity - intensity_min) / (intensity_max - intensity_min)
    else:
        intensity_norm = intensity
    
    features = np.column_stack((
        x_norm, y_norm, z_norm,
        r_norm, g_norm, b_norm,
        intensity_norm
    ))
    
    labels_reshaped = labels.reshape(-1, 1)
    dataset = np.hstack((features, labels_reshaped))
    
    np.save(output_file, dataset)
    
    plt.figure(figsize=(10, 6))
    unique_classes, counts = np.unique(labels, return_counts=True)
    plt.bar(unique_classes, counts)
    plt.xlabel('Класс')
    plt.ylabel('Количество точек')
    plt.title('Распределение меток по классам')
    plt.xticks(unique_classes)
    plt.grid(True, alpha=0.3)
    plt.savefig('class_distribution.png')
    plt.close()
    
    return dataset

dataset = process_semantic3d_data(
    "bildstein_station3_xyz_intensity_rgb.txt",
    "bildstein_station3_xyz_intensity_rgb.labels"
)

print("Первые 5 строк массива:")
print(dataset[:5])
print("\nФорма итогового массива:", dataset.shape)
print("Тип данных:", dataset.dtype)

Первые 5 строк массива:
[[-0.07775801 -2.07133174 -0.5946238  -0.52549022  0.50980395  0.57254905
   0.63529414  0.        ]
 [-0.07759166 -2.07160068 -0.65121341 -0.34901962  0.55686277  0.6156863
   0.69803923  0.        ]
 [-0.07767483 -2.07180238 -0.67875969 -1.10196078  0.56470591  0.6156863
   0.6901961   0.        ]
 [-0.07767483 -2.07180238 -0.67965794 -0.63529414  0.56078434  0.61176473
   0.68627453  0.        ]
 [-0.07775801 -2.07180238 -0.67965794 -0.44313726  0.56078434  0.61176473
   0.68627453  0.        ]]

Форма итогового массива: (23995481, 8)
Тип данных: float64


## Ответы на контрольные вопросы

1. **Дайте определение датасету Semantic3D. Каковы его основные характеристики и для каких задач он предназначен?**
Semantic3D — это датасет для семантической сегментации 3D облаков точек, полученных с помощью наземного лазерного сканирования (LiDAR). Содержит миллиарды точек outdoor-сцен (улицы, здания, парки) с 8 классами объектов. Предназначен для обучения моделей 3D-сегментации, распознавания объектов в автономных системах и роботах.

2. **Чем Semantic3D принципиально отличается от датасетов для 2D-компьютерного зрения и других 3D-датасетов для помещений?**
Отличается от 2D-датасетов (ImageNet) тем, что работает с 3D-координатами вместо пикселей. От indoor-3D датасетов (S3DIS) отличается тем, что содержит outdoor-сцены с большими площадями, имеет неравномерную плотность точек из-за особенностей наземного сканирования и включает другие объекты (дороги, деревья, автомобили).

3. **Опишите состав и структуру датасета. На какие множества он разделен?**
Датасет состоит из 30 облаков точек, разделенных на обучающее (15 сцен) и тестовое (15 сцен) множества. Каждая сцена представлена файлом .txt с координатами, интенсивностью и цветами точек, и файлом .labels с метками классов для каждой точки.

4. **Каким способом были получены данные в Semantic3D?**
Данные получены с помощью наземного лазерного сканирования (LiDAR), где сканеры устанавливались на неподвижные позиции или на автомобили. Это приводит к неравномерной плотности точек (высокая вблизи сканера, низкая на расстоянии) и наличию шума от отражений и ошибок измерений.

5. **Перечислите и охарактеризуйте 8 семантических классов в датасете.**
    1. Мост (bridge) - пешеходные и автомобильные мосты
    2. Здание (building) - жилые и коммерческие здания
    3. Дорога (road) - проезжие части, тротуары
    4. Столбы (pole) - фонарные столбы, дорожные знаки
    5. Земля (ground) - почва, трава
    6. Растения (vegetation) - деревья, кусты
    7. Пешеход (human) - люди на улицах
    8. Автомобиль (car) - легковые и грузовые автомобили

6. **Каковы основные проблемы, связанные с разметкой данных?**
Основные проблемы: субъективность при определении границ объектов, наличие шума и артефактов сканирования, высокая трудоемкость ручной разметки миллиардов точек, несогласованность разметки между разными аннотаторами, сложности с разметкой точек на границах объектов.

7. **Назовите основные технические проблемы при работе с облаками точек такого объема.**
Огромные требования к памяти (RAM и GPU), медленная обработка из-за объема данных, сложности визуализации, необходимость специальных структур данных, проблемы с хранением и передачей данных.

8. **Что такое "неравномерная плотность" точек в контексте LiDAR-данных?**
Неравномерная плотность — это изменение количества точек на единицу площади в зависимости от расстояния до сканера. Приводит к тому, что модели лучше обучаются на объектах с высокой плотностью точек и хуже — на объектах с низкой плотностью.

9. **Опишите проблему несбалансированности классов в Semantic3D.**
Некоторые классы (земля, здания, растения) представлены тысячами точек, а другие (люди, столбы) — единицами или десятками. Это приводит к тому, что модели склонны предсказывать частые классы, игнорируя редкие, а метрика общей точности становится обманчиво высокой.

10. **Какие основные этапы предобработки данных необходимы?**
Основные этапы: нормализация координат (центрирование и масштабирование), нормализация признаков (интенсивность, RGB), даунсэмплинг для уменьшения объема данных, удаление шума, балансировка классов и аугментация данных (вращение, сдвиг).